In [1]:
import qlib
from qlib.constant import REG_CN
provider_uri = "../data/qlib_data/cn_data_ak"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

[16756:MainThread](2022-08-20 19:25:06,201) INFO - qlib.Initialization - [config.py:413] - default_conf: client.
[16756:MainThread](2022-08-20 19:25:06,916) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[16756:MainThread](2022-08-20 19:25:06,917) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('D:/PycharmProjects/QlibPlatform/data/qlib_data/cn_data_ak')}


In [ ]:
# 导入QlibDataLoader
from qlib.data.dataset.loader import QlibDataLoader

In [4]:
# 加载原始特征，比如收盘价、最高价
qdl = QlibDataLoader(config=(['$close', '$high', '$factor'],['close', 'high', 'factor'])) 
df = qdl.load(instruments=['idx399006'], start_time='20190101', end_time='20221231')

In [21]:
# 这次我们加载沪深300成分股的10日和30日收盘价指数加权均价
market = 'csi300' # 沪深300股票池代码，在instruments文件夹下有对应的sh000300.txt
close_ma = ['EMA($close, 10)', 'EMA($close, 30)'] # EMA($close, 10)表示计算close的10日指数加权均线
ma_names = ['EMA10', 'EMA30']
qdl_ma = QlibDataLoader(config=(close_ma, ma_names)) 
df = qdl_ma.load(instruments=market, start_time='20210101', end_time='20210110')

In [10]:
market = 'csi300' # 沪深300股票池代码，在instruments文件夹下有对应的sh000300.txt
close_ma = ['EMA($close, 10)', 'EMA($close, 30)'] # EMA($close, 10)表示计算close的10日指数加权均线
ma_names = ['EMA10', 'EMA30']
ret = ["Ref($close, -1)/$close-1"] # 下一日收益率, Ref($close, -1)表示下一日收盘价
ret_name = ['next_ret']
qdl_ma_gp = QlibDataLoader(config={'feature':(close_ma, ma_names), 'label': (ret, ret_name)}) 
qdl_ma_gp.load(instruments=market, start_time='20210101', end_time='20210110')

feature                 label
                            EMA10       EMA30  next_ret
datetime   instrument                                  
2021-01-04 SH600000     87.085709   87.898911 -0.000799
           SH600004     28.742598   29.356647 -0.013638
           SH600009    185.496445  188.231216 -0.002277
           SH600010      9.318274    9.363085 -0.007463
           SH600011     17.678295   17.891283 -0.011396
...                           ...         ...       ...
2021-01-08 SZ300498     38.887028   38.799690 -0.033316
           SZ300529    143.827393  138.928024 -0.046978
           SZ300601    250.708481  245.788208 -0.062018
           SZ300628    889.865051  876.632812 -0.008940
           SZ300676    133.755264  132.078094  0.053054

[1495 rows x 3 columns]

In [26]:
from qlib.data.filter import ExpressionDFilter
# QlibDataLoader其他参数: filter_pipe
market = 'csi300' # 沪深300股票池代码，在instruments文件夹下有对应的sh000300.txt
close_ma = ['EMA($close, 10)', 'EMA($close, 30)'] # EMA($close, 10)表示计算close的10日指数加权均线
ma_names = ['EMA10', 'EMA30']

# 使用表达式定义过滤规则
filter_rule = ExpressionDFilter(rule_expression='EMA($close, 10)>EMA($close, 30)')

# 导出数据
qdl_fil = QlibDataLoader(config=(close_ma, ma_names), filter_pipe=[filter_rule,]) 
qdl_fil.load(instruments=market, start_time='20210101', end_time='20220110')

EMA10        EMA30
datetime   instrument                          
2021-01-04 SH600018      18.029676    17.958061
           SH600025       5.010724     5.018754
           SH600031    1367.375366  1316.675049
           SH600111     219.184631   217.240051
           SH600161     373.412048   368.703583
...                            ...          ...
2022-01-10 SZ300628    1009.209351  1000.033386
           SZ300676      91.195946    91.037697
           SZ300677     274.691650   272.923492
           SZ300888      86.108780    84.384277
           SZ300999      64.652382    63.982834

[31780 rows x 2 columns]

In [46]:
from qlib.data.dataset.handler import DataHandlerLP
from qlib.data.dataset.processor import CSZScoreNorm, DropnaProcessor

qdl = QlibDataLoader(config=(['$close/Ref($close, 1)-1'],['Return']))
df = qdl.load(instruments='csi300', start_time='20210101', end_time='20210430') 

In [47]:
# 是否有空值
df.isna().sum()

Return    31
dtype: int64

In [49]:
# 实例化DataHandler
dh = DataHandlerLP(instruments='csi300', start_time='20210101', end_time='20210120',
             learn_processors=[DropnaProcessor(), CSZScoreNorm()],
             data_loader=qdl)

[16756:MainThread](2022-08-20 21:14:01,978) INFO - qlib.timer - [log.py:117] - Time cost: 11.806s | Loading data Done
[16756:MainThread](2022-08-20 21:14:01,981) INFO - qlib.timer - [log.py:117] - Time cost: 0.002s | DropnaProcessor Done
[16756:MainThread](2022-08-20 21:14:02,000) INFO - qlib.timer - [log.py:117] - Time cost: 0.018s | CSZScoreNorm Done
[16756:MainThread](2022-08-20 21:14:02,001) INFO - qlib.timer - [log.py:117] - Time cost: 0.022s | fit & process data Done
[16756:MainThread](2022-08-20 21:14:02,001) INFO - qlib.timer - [log.py:117] - Time cost: 11.830s | Init data Done
